# Why?
I had a debate with a friend about *Bones*, a (very) long-running TV show. She claimed that it was as popular as ever, which was why it was still on air; I figured that while still popular, there was no way that it still drew the same number of viewers that it used to.

As with most arguments these days, we went to Wikipedia (https://en.wikipedia.org/wiki/List_of_Bones_episodes) to find our answers. Fortunately, they had viewership numbers, so we were able to settle the argument (I was right). Unfortunately, it's just a big table, so I couldn't show her just how far the show's numbers have fallen.

In order to do that, I needed to grab all of those numbers in a graphing-friendly format, which means:
1. Scraping the data
2. Cleaning it
3. Writing it out

*To do, eventually: preliminary graphing/exploration in this notebook*

## 1. Import and Scrape

In [1]:
import requests
from lxml import html
import pandas as pd

In [2]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_Bones_episodes")
tree = html.fromstring(page.content.decode('utf-8'))

With lxml, you need to use XPath to get the right selection from the DOM.
The XPath below does this:
* Find the `span` element with an `id` property whose value is "Episodes"
* From that element, go to its parent - specifically, the `h2` that contains it
* From the `h2`, within the set of its following siblings, find the `table` nodes
* Find the child `tr`s within those `table`s

In [3]:
rows = tree.xpath(
    '//span[@id="Episodes"]/parent::h2/following-sibling::table/tr'
    )

In order to instantiate a DataFrame containing our data, I need to extract the content I want and put it in a data structure. A list will work here.

In [4]:
data = []
for row in rows:  # Iterate through all tr elements
    # Add lists consisting of the text content of every child element of the current tr - i.e. every cell
    data.append([c.text_content().__str__() for c in row.getchildren()])

In [5]:
episodes = pd.DataFrame(data)
episodes.head()  # See the first five rows to get a sense of what's in it

,0,1,2,3,4,5,6,7
0,No. in\nseries,No. in\nseason,Title,Directed by,Written by,Original air date,Production\ncode,U.S. viewers\n(millions)
1,1,1,"""Pilot""",Greg Yaitanes,Hart Hanson,"September 13, 2005 (2005-09-13)",1AKY79,10.79[1]
2,2,2,"""The Man in the S.U.V.""",Allan Kroeker,Stephen Nathan,"September 20, 2005 (2005-09-20)",1AKY02,7.39[2]
3,3,3,"""A Boy in a Tree""",Patrick Norris,Hart Hanson,"September 27, 2005 (2005-09-27)",1AKY01,7.87[3]
4,4,4,"""The Man in the Bear""",Allan Kroeker,Laura Wolner,"November 1, 2005 (2005-11-01)",1AKY04,7.99[4]


## 2. Clean

Oops! Looks like there are a few problems I can see right away:
1. The columns have no names
2. The first row of the table (index 0) consists of column names (since the first `tr` element contained the headers)
3. If I use row 0 as our column names, I probably don't want those newlines (`\n`) in there

What I'll want to do:
1. Replace newlines in row 0
2. Set the column labels to be equal to corresponding values in row 0
3. Get rid of row 0

In [6]:
# 1. Replace newlines
episodes.iloc[0] = episodes.iloc[0].str.replace('\n', ' ')
# And check to make sure it did what was expected
episodes.iloc[0]

0              No. in series
1              No. in season
2                      Title
3                Directed by
4                 Written by
5          Original air date
6            Production code
7    U.S. viewers (millions)
Name: 0, dtype: object

In [7]:
# 2. Set column names to values of the first row
episodes.columns = episodes.iloc[0]
episodes.head()

,No. in series,No. in season,Title,Directed by,Written by,Original air date,Production code,U.S. viewers (millions)
0,No. in series,No. in season,Title,Directed by,Written by,Original air date,Production code,U.S. viewers (millions)
1,1,1,"""Pilot""",Greg Yaitanes,Hart Hanson,"September 13, 2005 (2005-09-13)",1AKY79,10.79[1]
2,2,2,"""The Man in the S.U.V.""",Allan Kroeker,Stephen Nathan,"September 20, 2005 (2005-09-20)",1AKY02,7.39[2]
3,3,3,"""A Boy in a Tree""",Patrick Norris,Hart Hanson,"September 27, 2005 (2005-09-27)",1AKY01,7.87[3]
4,4,4,"""The Man in the Bear""",Allan Kroeker,Laura Wolner,"November 1, 2005 (2005-11-01)",1AKY04,7.99[4]


In [8]:
# 3. Get rid of row 0
episodes.drop(0, inplace=True)
episodes.head()

,No. in series,No. in season,Title,Directed by,Written by,Original air date,Production code,U.S. viewers (millions)
1,1,1,"""Pilot""",Greg Yaitanes,Hart Hanson,"September 13, 2005 (2005-09-13)",1AKY79,10.79[1]
2,2,2,"""The Man in the S.U.V.""",Allan Kroeker,Stephen Nathan,"September 20, 2005 (2005-09-20)",1AKY02,7.39[2]
3,3,3,"""A Boy in a Tree""",Patrick Norris,Hart Hanson,"September 27, 2005 (2005-09-27)",1AKY01,7.87[3]
4,4,4,"""The Man in the Bear""",Allan Kroeker,Laura Wolner,"November 1, 2005 (2005-11-01)",1AKY04,7.99[4]
5,5,5,"""A Boy in a Bush""",Jesús Treviño,Steve Blackman & Greg Ball,"November 8, 2005 (2005-11-08)",1AKY05,6.86[5]


When I was looking at the Wikipedia page, I also noticed some weird stuff at the bottom of the table. Better double-check.

In [9]:
episodes.tail(15)

,No. in series,No. in season,Title,Directed by,Written by,Original air date,Production code,U.S. viewers (millions)
235,225,13,"""The Monster in the Closet""[220]",TBA,Michael Peterson,TBA,BAKY13,TBA
236,226,14,"""The Last Shot at a Second Chance""[221]",David Grossman[222],Emily Silver,TBA,BAKY14,TBA
237,Season,Episodes,DVD release dates,Blu-ray release dates,None,None,None,None
238,Region 1,Region 2,Region 4,Region A,Region B,None,None,None
239,1,22,"November 28, 2006[223]","October 30, 2006[224]","January 11, 2007[225]",N/A,N/A,None
240,2,21,"September 11, 2007[226]","October 15, 2007[227]","December 3, 2008[228]",N/A,N/A,None
241,3,15,"November 18, 2008[229]","November 17, 2008[230]","March 4, 2009[231]",N/A,N/A,None
242,4,26,"October 6, 2009[232]","October 26, 2009[233]","October 28, 2009[234]","October 6, 2009[232]",N/A,None
243,5,22,"October 5, 2010[235]","October 18, 2010[236]","October 27, 2010[237]","October 5, 2010[235]","October 18, 2010[238]",None
244,6,23,"October 11, 2011[239]","October 17, 2011[240]","November 9, 2011[241]","October 11, 2011[239]","October 17, 2011[242]",None


It appears that the scraping script grabbed the "home media release" section, as well as some unaired episodes. I'll drop those rows, since they have no value for my purposes. They can be identified by having a `None` or "TBA" value in the viewers column.

In [10]:
# The isnull method checks for None
episodes.drop(episodes[episodes['U.S. viewers (millions)'].isnull()].index, inplace=True)
episodes.drop(episodes[episodes['U.S. viewers (millions)'] == "TBA"].index, inplace=True)
episodes.tail(15)

,No. in series,No. in season,Title,Directed by,Written by,Original air date,Production code,U.S. viewers (millions)
218,209,19,"""The Murder in the Middle East""",Milan Cheylov,Michael Peterson,"May 14, 2015 (2015-05-14)",AAKY19,4.71[205]
219,210,20,"""The Woman in the Whirlpool""",Dwight Little,Kathy Reichs & Kerry Reichs,"May 28, 2015 (2015-05-28)",AAKY20,5.42[206]
220,211,21,"""The Life in the Light""",Randy Zisk,Keith Foglesong,"June 4, 2015 (2015-06-04)",AAKY21,5.21[207]
221,212,22,"""The Next in the Last""",Ian Toynton,Stephen Nathan & Jonathan Collier,"June 11, 2015 (2015-06-11)",AAKY22,5.11[208]
222,No. in\nseries,No. in\nseason,Title,Directed by,Written by,Original air date,Production\ncode,U.S. viewers\n(millions)
223,213,1,"""The Loyalty in the Lie""",Randy Zisk,Jonathan Collier,"October 1, 2015 (2015-10-01)",BAKY01,6.20[209]
224,214,2,"""The Brother in the Basement""",Dwight Little,Michael Peterson,"October 8, 2015 (2015-10-08)",BAKY02,5.90[210]
225,215,3,"""The Donor in the Drink""",Michael Lange,Hilary Weisman Graham,"October 15, 2015 (2015-10-15)",BAKY03,5.79[211]
226,216,4,"""The Carpals in the Coy-Wolves""",Randy Zisk,Gene Hong,"October 22, 2015 (2015-10-22)",BAKY04,6.06[212]
227,217,5,"""The Resurrection in the Remains""",Chad Lowe,Mary Trahan,"October 29, 2015 (2015-10-29)",BAKY05,6.57[213]


Another thing I didn't notice the first time: since there are actually many HTML tables, each one has its own header row. These were scraped into the list as well (see row 222), so I'll delete those.

In [11]:
episodes.shape

(232, 8)

In [12]:
# Header rows, in this case, can be identified by a column value being equal to its own header
episodes.drop(episodes[episodes['Title'] == 'Title'].index, inplace=True)
episodes.shape

(222, 8)

I'll also pull out just the date part of the air date. It'll be easiest to grab the dates in `YYYY-MM-DD` format from between the parentheses.

In [13]:
episodes.loc[:, 'Date'] = pd.to_datetime(episodes['Original air date'].str[-11:-1])
episodes['Date'].head()

1   2005-09-13
2   2005-09-20
3   2005-09-27
4   2005-11-01
5   2005-11-08
Name: Date, dtype: datetime64[ns]

I'm also going to have to remove the citations in the Viewers column. I'll create a function that uses regex to find the citation number in brackets and remove it. Then, `map` that function to all rows.

In [14]:
import re
viewers_regex = re.compile(r'\[\d*\]')  # Any number of digits between square brackets

def strip_viewers(text):
    viewer_match = re.search(viewers_regex, text)
    if viewer_match is not None:
        return float(text.replace(viewer_match.group(0), ''))        
    else:
        return None
    
episodes.loc[:, 'Viewers (M)'] = episodes['U.S. viewers (millions)'].map(strip_viewers).astype(float)
episodes['Viewers (M)'].tail()

228    5.34
229    5.16
230    5.25
231    4.63
232    4.42
Name: Viewers (M), dtype: float64

Let's take a look at some quick summary statistics.

In [15]:
episodes['Viewers (M)'].describe()

count    222.000000
mean       8.428919
std        1.962348
min        4.290000
25%        7.052500
50%        8.630000
75%        9.870000
max       12.640000
Name: Viewers (M), dtype: float64

## 3. Write to File

Now, I'll create a new DataFrame to hold just the columns I care about, and export those to a text file.

In [16]:
episodes.iloc[:, [0, 1, 2, 3, 8, 9]].to_csv('bones_episodes_cleaned.txt', sep='\t', index=False)